## Preditor de Evasão Estudantil
Este projeto pretende identificar os estudantes da graduação da Universidade de Brasília com risco de evasão a partir do histórico acadêmico. Leia atentamente este guia para executar os algoritmos.

### Como usar
Primeiro, é necessário adicionar os dados dos estudantes a este notebook. Para isso, clique no botão `Copy and Edit`, que o levará a uma página para edição do notebook. Nessa página, clique no botão `+ Add Data`. São necessários três arquivos CSV para rodar os algoritmos: `training_data.csv`, `test_data.csv` e `materias.csv`. Criei um arquivo ZIP com os três arquivos e faça o upload.

#### training_data.csv
O `training_data.csv` contém os dados dos estudantes que já terminaram o curso, evadidos ou formados. O formato do `training_data.csv` segue abaixo:

| IdAluno | SemestreIngresso | SemestreMateria |  CodigoMateria | Conceito | StatusFinal |
|---------|------------------|-----------------|----------------|----------|-------------|
| 1234    | 20101            | 20102           |  113034        | "SS"     | "FORMADO"   |
| 1235    | 20112            | 20121           |  118028        | "II"     | "EVADIDO"   |

+ **IdAluno** (*inteiro*): identificador único de um aluno
+ **SemestreIngresso** (*inteiro*): o semestre de ingresso do aluno no curso de graduação. Esse campo deve seguir o formato `AAAAS`, onde `A` é um dígito do ano e `S` é o semestre, e.g. '20101' para um aluno que ingressou no 1º semestre de 2010.
+ **SemestreMateria** (*inteiro*): o semestre no qual o aluno cursou determinada matéria. Esse campo deve seguir o formato `AAAAS`, onde `A` é um dígito do ano e `S` é o semestre.
+ **CodigoMateria** (*inteiro*): o código da matéria. Verifique os códigos das matérias no [Matrícula Web](https://matriculaweb.unb.br/).
+ **Conceito** (*string*): menção do aluno naquela matéria no semestre.
+ **StatusFinal** (*string*): o status do aluno ao final do curso. Os possíveis valores para este campo devem ser:
  + Se formado => `FORMADO`
  + Se evadido => `EVADIDO`

No exemplo acima, o estudante 1234, que formou, entrou no 1º semestre de 2010 e cursou a matéria com código 113034 (Cálculo 1) no 2º semestre de 2010 e obteve a menção SS. A segunda linha mostra que o estudante evadido 1235 entrou no 2º semestre de 2011 e cursou 118028 (Física 2) no 1º semestre de 2012 e obteve II.

#### test_data.csv
O arquivo `test_data` contém os dados dos alunos que estão ativos e que desejam ser classificados em evadidos ou formados. Esse arquivo tem o mesmo formato do `training_data.csv` excetuando a coluna `StatusFinal`, que não está presente.

#### materias.csv
O arquivo `materias.csv` tem as matérias contidas nos dados de treinamento e teste. **O algoritmo utilizará as matérias que estão nesse arquivo. Recomenda-se usar as matérias obrigatórias dos dois primeiros semestres do curso.** O formato do arquivo é:

| CodigoMateria | Creditos |
|---------------|----------|
| 113034        | 6        |
| 118028        | 4        |

#### Observações
+ Alunos que vieram a falecer não devem ser usados na entrada.
+ Todos os estudantes devem estar matriculados no mesmo curso de graduação.


In [ ]:
import pandas as pd

input_train_df = pd.read_csv('../input/mecatronics-bi-dump/training_data.csv')
input_train_df.head()

Dataframe para o treinamento dos algoritmos de classificação

In [ ]:
materias_df = pd.read_csv('../input/mecatronics-bi-dump/materias.csv')

def calculate(entry_semester, course_semester):
    if is_summer_course(course_semester):
        course_semester += 1

    diff = course_semester - entry_semester

    if entry_semester % 2 != 0 :
        if diff >= 10 :
            result = semesters_between_years(diff)
        else:
            result = diff
    else:
        if diff % 10 == 0 :
            result = semesters_between_years(diff)
        else:
            result = diff // 5

    return result + 1

def is_summer_course(semester):
    return semester % 10 == 0

def semesters_between_years(diff):
    return (diff // 5) + (diff % 5)

def transform_dataframe(df, aggfunc, fill_value, groupby_columns):
    df['CourseTerm'] = df.Semester.map(str) + '_' + df.CodigoMateria.map(str)
    df = df.drop(columns=['SemestreIngresso', 'SemestreMateria', 'CodigoMateria', 'Semester'])

    df = df.pivot_table(values='Conceito', index=groupby_columns, columns='CourseTerm', aggfunc=aggfunc, fill_value=fill_value)
    df.columns.name = None
    return df.reset_index()

def failed_workload(row, semester=None):
    failed_workload = 0

    for code, workload in list(zip(materias_df['CodigoMateria'], materias_df['Creditos'])):
        if semester == None:
            if '1_' + str(code) in row: failed_workload += row['1_' + str(code)] * workload
            if '2_' + str(code) in row: failed_workload += row['2_' + str(code)] * workload
        else:
            if semester + '_' + str(code) in row: failed_workload += row[semester + '_' + str(code)] * workload

    return failed_workload

def calculate_semester(df):
    df['Semester'] = df.apply(lambda x: calculate(x['SemestreIngresso'], x['SemestreMateria']), axis=1)
    return df[(df.Semester > 0) & (df.Semester <= 2)]

def one_hot_encoding(df, groupby_columns):
    columns = df.columns.difference(groupby_columns).tolist()
    for column in columns:
        one_hot = pd.get_dummies(df[column])
        df = df.drop(column,axis = 1)
        one_hot.columns = map(lambda x: column + '_' + x, one_hot.columns)
        df = df.join(one_hot)
    return df

def calculate_failed_workload(df, aux_df, groupby_columns):
    aux_df['Conceito'] = aux_df['Conceito'].replace(['SR', 'II', 'MI'], 1)
    aux_df['Conceito'] = aux_df['Conceito'].replace(['SS', 'MS', 'MM', 'CC', 'DP', 'TR', 'TJ'], 0)
    aux_df = transform_dataframe(aux_df, 'sum', 0, groupby_columns)
    
    df['Creditos_Reprovados'] = aux_df.apply(lambda row: failed_workload(row), axis=1)
    df['Creditos_Reprovados_1'] = aux_df.apply(lambda row: failed_workload(row, '1'), axis=1)
    df['Creditos_Reprovados_2'] = aux_df.apply(lambda row: failed_workload(row, '2'), axis=1)
    
    return df

def model_data(df, groupby_columns=['IdAluno', 'StatusFinal']):
    aux_df = df.copy()
    df = transform_dataframe(df, 'last', 'NC', groupby_columns)
    df = one_hot_encoding(df, groupby_columns)
    df = calculate_failed_workload(df, aux_df, groupby_columns)
    
    return df

input_train_df = calculate_semester(input_train_df)
train_df = model_data(input_train_df)
train_df.head()

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

logreg_param_grid = {
    'solver': ['liblinear', 'lbfgs'],
    'C': np.logspace(-2, 4, 10)
}


dtree_param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True)
}

rf_param_grid = {
    'criterion': ['gini', 'entropy'],
    'n_estimators': range(10,200,20),
    'max_features': ['auto', 'sqrt']
}

classifiers = [
    ('LogisticRegression', LogisticRegression(max_iter=1500), logreg_param_grid),
    ('DecisionTreeClassifier', tree.DecisionTreeClassifier(), dtree_param_grid),
    ('RandomForestClassifier', RandomForestClassifier(), rf_param_grid)
]

feature_cols = train_df.columns.difference(['StatusFinal', 'IdAluno'])
features = train_df.loc[:, feature_cols]
labels = train_df.StatusFinal.replace({'EVADIDO': 1, 'FORMADO': 0})
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.30, stratify=labels, random_state=42)

scaler.fit(X_train.values)
X_train = scaler.transform(X_train.values)
X_test = scaler.transform(X_test.values)

Treinamento e validação dos classificadores:
- [sensibilidade](https://en.wikipedia.org/wiki/Precision_and_recall#Recall): porcentagem dos estudantes que realmente vão evadir que foram classificados corretamente
- [acurácia](https://en.wikipedia.org/wiki/Accuracy_and_precision): habilidade do algoritmo de classificar os estudantes corretamente

Como o objetivo é encontrar o maior número possível de estudantes que realmente vão evadir, treinamos os classificadores para **maximizar a sensibilidade**.

In [ ]:
best_classifiers = []
for classifier in classifiers:
    grid_search = GridSearchCV(classifier[1], classifier[2], scoring='recall',
                          cv=10, return_train_score=True, iid=True)
    grid_search.fit(X_train, y_train.values)

    y_pred = grid_search.predict(X_test)
    print(classifier[0])
    print('Melhores parametros para sensibilidade', grid_search.best_params_)
    print("sensibilidade = %0.4f" % recall_score(y_test.values, y_pred))
    print("acurácia = %0.4f" % accuracy_score(y_test.values, y_pred))
    best_classifiers.append(grid_search.best_estimator_)

Agora lemos os dados de treinamento e fazemos as previsões.

In [ ]:
test_df = pd.read_csv('../input/mecatronics-bi-dump/test_data.csv')
test_df = calculate_semester(test_df)
test_df = model_data(test_df, ['IdAluno'])
remaining_columns = train_df.columns.difference(test_df.columns).tolist()
remaining_columns.remove('StatusFinal')
for column in remaining_columns:
    test_df[column] = 0
test_df.head()

In [ ]:
feature_cols = test_df.columns.difference(['IdAluno'])
features = test_df.loc[:, feature_cols]

predictions = []
for classifier in best_classifiers:
    predictions.append(classifier.predict(features))

results_df = pd.DataFrame(columns=['IdAluno', 'Regressao Logistica', 'Arvores de Decisao', 'Florestas Aleatorias'])
index = 0
for student in range(test_df.shape[0]):
    results_df.loc[index] = [test_df['IdAluno'][index]] + \
                                ['EVADIDO' if predictions[0][index] else 'FORMADO'] + \
                                ['EVADIDO' if predictions[1][index] else 'FORMADO'] + \
                                ['EVADIDO' if predictions[2][index] else 'FORMADO']
    index += 1

results_df

## Regras de associação
O aprendizado de regras de associação é um método de aprendizado de máquina baseado em regras para descobrir relações interessantes entre variáveis ​​em grandes bancos de dados. [Mais informações](https://en.wikipedia.org/wiki/Association_rule_learning).

In [ ]:
association_df = input_train_df.copy()

association_df = association_df.drop(columns=['SemestreIngresso', 'SemestreMateria'])
association_df = association_df.applymap(str)

association_df['TermCourseGrade'] = association_df.Semester + '_' + association_df.CodigoMateria + '_' + association_df.Conceito
association_df = association_df.drop(columns=['Conceito', 'CodigoMateria', 'Semester'])

grouped = association_df.groupby(['IdAluno', 'StatusFinal'])
association_df = grouped['TermCourseGrade'].apply(lambda x: pd.Series(x.values)).unstack()
association_df = association_df.reset_index()

association_df = association_df.drop(columns=['IdAluno'])
association_df.head()

In [ ]:
from apyori import apriori

records = association_df.T.apply(lambda x: x.dropna().tolist()).tolist()
rules = apriori(records, min_support=0.03, min_confidence=0.8)
rules_df = pd.DataFrame(columns=('Antecedent','Consequent','Support','Confidence'))
Support =[]
Confidence = []
Antecedent = []
Consequent=[]

for RelationRecord in rules:
    for ordered_stat in RelationRecord.ordered_statistics:
        Support.append(RelationRecord.support)
        Antecedent.append(list(ordered_stat.items_base))
        Consequent.append(ordered_stat.items_add)
        Confidence.append(ordered_stat.confidence)
                             
rules_df['Antecedent'] = list(Antecedent)
rules_df['Consequent'] = Consequent
rules_df['Support'] = Support
rules_df['Confidence'] = Confidence

rules_df.sort_values(by =['Confidence', 'Support'], ascending = False, inplace = True)

rules_df = rules_df[rules_df['Consequent'] == {'EVADIDO'}]

for index, row in rules_df.iterrows():
    antecedents = [antecedent.split('_') for antecedent in row.Antecedent]
    for i, antecedent in enumerate(antecedents, start=0):
        if i == 0: print('se tirou ', end='')
        else: print(' e tirou ', end='')
        print('%s em %s no %sº semestre' % (antecedent[2],antecedent[1],antecedent[0]), end='')
    print(', então será %s com %s%% de confiança => %s casos' % (list(row.Consequent)[0], round(row.Confidence * 100, 2), round(row.Support * association_df.shape[0], 0)))


Aqui temos as regras de associação obtidas a partir dos dados de treinamento, ou seja, dos dados do passado do curso. Por exemplo, uma linha

| Se          | Entao   | Confiança | Ocorrencias |
|-------------|---------|-----------|-------------|
| 1_113034_II | EVADIDO | 96.153846 | 50.0        |

significa que se um aluno tiver tirado II em 113034 (Cálculo 1) no seu 1º semestre, ele tem chances de evadir com 96% de confiança e isso aconteceu 50 vezes nos dados.

[**Confiança**](https://en.wikipedia.org/wiki/Association_rule_learning#Confidence) é uma indicação de quantas vezes a regra foi considerada verdadeira. No nosso exemplo, é a proporção de vezes que estudantes tinham 1_113034_II no seu histórico e evadiram sobre a quantidade de vezes que alunos tinham 1_113034_II.

In [ ]:
rules_df.head()